## Import

In [1]:
import glob
import torch
from torch.utils import data
from PIL import Image
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from ahrs.filters import Madgwick
from ahrs.common.orientation import q2euler
from PIL import Image, ImageDraw
import os

#通过创建data.Dataset子类Mydataset来创建输入
class Mydataset(data.Dataset):
# 类初始化
    def __init__(self, root):
        self.imgs_path = root
# 进行切片
    def __getitem__(self, index):
        img_path = self.imgs_path[index]
        return img_path
# 返回长度
    def __len__(self):
        return len(self.imgs_path)


/home/lyc/anaconda3/envs/mix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Go through every image in the folder

In [10]:
#使用glob方法来获取数据图片的所有路径
all_imgs_path = glob.glob(r'/home/lyc/下载/signature_xizhi/*.csv')#数据文件夹路径，根据实际情况更改！
new_img_dir = '/home/lyc/doc/AML_LAB/img_data'  
new_imu_dir = '/home/lyc/doc/AML_LAB/imu_data'  
#循环遍历输出列表中的每个元素，显示出每个图片的路径


#利用自定义类Mydataset创建对象weather_dataset
signature_dataset = Mydataset(all_imgs_path)
print(len(signature_dataset)) #返回文件夹中图片总个数
print(signature_dataset[12:15])#切片，显示第12至第十五张图片的路径
sinature_datalodaer = torch.utils.data.DataLoader(signature_dataset, batch_size=5) #每次迭代时返回五个数据
#print(next(iter(sinature_datalodaer)))




20
['/home/lyc/下载/signature_xizhi/True_9.csv', '/home/lyc/下载/signature_xizhi/False_3.csv', '/home/lyc/下载/signature_xizhi/True_6.csv']


## Transform IMU

In [17]:
species = ['False','True']
species_to_id = dict((c, i) for i, c in enumerate(species))
#print(species_to_id)
id_to_species = dict((v, k) for k, v in species_to_id.items())
#print(id_to_species)
all_labels = []
#对所有图片路径进行迭代
for imu in all_imgs_path:
    # 区分出每个img，应该属于什么类别
    for i, c in enumerate(species):
        if c in imu:
            all_labels.append(i)
print(all_labels) #得到所有标签 


for var in all_imgs_path:
    data = pd.read_csv(var)
    x_coords=data.values[0::,1]
    y_coords=data.values[0::,2]
    pressure=data.values[0::,3]
    acc_data1 = data[['accel_x', 'accel_y', 'accel_z']].to_numpy()
    gyro_data1 = data[['gyro_x', 'gyro_y', 'gyro_z']].to_numpy()
    # 初始化Madgwick过滤器，这里假设数据采样率为50Hz
    delta_t = 1/50
    madgwick = Madgwick(delta_t=delta_t)
    q = [1, 0, 0, 0]
    # 使用加速度计和陀螺仪数据处理IMU数据，估算姿态
    # 不需要显式指定初始四元数，Madgwick过滤器会自动处理
    quaternions = np.array([madgwick.updateIMU(q=q,acc=acc_data1[i], gyr=gyro_data1[i]) for i in range(len(acc_data1))])

    # 将四元数转换为欧拉角，以便更容易解释
    euler_angles = np.array([q2euler(q) for q in quaternions])
    euler_angles_df = pd.DataFrame(euler_angles, columns=['roll', 'pitch', 'yaw'])

    combined_data = pd.concat([data[['x', 'y', 'pressure']], euler_angles_df], axis=1)

      
    base_name = os.path.basename(var)  # 获取原始文件的基本名称
    new_imu_name = os.path.splitext(base_name)[0] + '.csv'  # 更改文件扩展名为.png
    new_imu_path = os.path.join(new_imu_dir, new_imu_name)  # 创建新的文件路径
    # 保存到新的CSV文件
    combined_data.to_csv(new_imu_path, index=False)


    combined_data.head()


[1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0]


## Transform IMG

In [15]:
species = ['False','True']
species_to_id = dict((c, i) for i, c in enumerate(species))
id_to_species = dict((v, k) for k, v in species_to_id.items())
all_labels = []
#对所有图片路径进行迭代
for imu in all_imgs_path:
    # 区分出每个img，应该属于什么类别
    for i, c in enumerate(species):
        if c in imu:
            all_labels.append(i)



for var in all_imgs_path:
    data = pd.read_csv(var)
    x_coords=data.values[0::,1]
    y_coords=data.values[0::,2]
    pressure=data.values[0::,3]
    pressure = pressure-np.mean(pressure)
# 创建一个白色背景的图像
    img = Image.new('RGB', (240, 320), color='white')
    draw = ImageDraw.Draw(img)


    max_pressure = np.max(pressure)

    for x, y, p in zip(x_coords, y_coords, pressure):
        #print(p)
        if p > 0:  # 只绘制压力大于0的点
        
            color = (int(p / max_pressure *255), 0, 0)  # 根据压力值生成颜色
            draw.point((x, y), fill=color)




    img = img.rotate(90)            
    base_name = os.path.basename(var)  # 获取原始文件的基本名称
    new_img_name = os.path.splitext(base_name)[0] + '.png'  # 更改文件扩展名为.png
    new_img_path = os.path.join(new_img_dir, new_img_name)  # 创建新的文件路径
    
    img.save(new_img_path)  # 保存图像到新的文件路径
    # 保存到新的CSV文件

[47 46 46 46 46 58 43 46 46 52 46 48 47 48 49 51 52 53 57 57 60 62 65 68
 71 75 75 80 85 87 90 91 92 92 93 93 93 92 91 89 87 85 82 81 80 76 80 79
 80 81 82 83 84 85 87 88 89 88 90 90 90 90 90 90 79 89 87 87 83 83 81 78
 77 76 74 73 83 70 70 69 69 67 68 66 66 66 66 65 66 66 67 67 67 67 67 66
 67 67 68 69 68 69 70 71 71 72 72 71 63 72 72 72 72 74 74 74 73 74 187 184
 193 119 200 197 196 118 200 190 197 122 199 192 261 204 197 197 198 201
 205 119 193 197 99 97 97 97 97 99 101 97 104 106 109 112 113 116 118 119
 121 122 122 123 181 200 196 193 194 206 199 115 194 252 198 198 195 205
 198 187 195 202 120 119 118 117 118 119 118 119 117 127 116 115 114 112
 111 109 107 105 104 103 97 101 100 99 99 99 99 99 99 101 102 104 108 111
 114 117 120 122 126 128 131 132 134 135 136 136 144 137 137 192 191 199
 201 199 218 198 200 201 191 113 138 124 122 122 121 122 121 120 120 116
 120 120 120 121 120 122 123 122 123 112 124 124 124 124 125 126 125 118
 126 126 125 126 126 133 125 126 125 125 124 12